In [6]:
diag(rand(3,3))

3-element Array{Float64,1}:
 0.8354053069372076
 0.2816786255332524
 0.20695380590534906

In [31]:

using StatsBase
using LinearAlgebra
using Distributions
include("FHMMs.jl")

Y = rand(5,5)
h = rand(3,5,2)
test = Main.FHMMs.FHMM(5,3,2)
for i in 1:5
    h = Main.FHMMs.update_variational_parameters(test, h, Y)
    display(h)
end
#test.W

"Delta: -5.4752533240903665"

3×5×2 Array{Float64,3}:
[:, :, 1] =
  0.411404   0.338703  0.515495   0.190966   0.276234
  0.721495   0.81861   0.661148   0.593799   0.744833
 -0.268544  -0.444232  0.216161  -0.468864  -0.482182

[:, :, 2] =
 0.389685  0.239556  0.530238  0.157207  0.230928
 0.567246  0.559326  0.5936    0.377111  0.513321
 0.400118  0.324     0.560598  0.196746  0.237252

"Delta: 10.599133009569393"

3×5×2 Array{Float64,3}:
[:, :, 1] =
 0.520486   0.792858   0.418316   1.0279    0.73263
 0.744273   0.838274   0.657368   0.896757  0.839521
 0.0212595  0.479414  -0.0576494  1.13616   0.366889

[:, :, 2] =
 0.437454  0.857564  0.387055  1.20127   0.711488
 0.630129  0.834087  0.544678  0.991834  0.824344
 0.571188  0.759616  0.460099  0.953378  0.72245

"Delta: -12.689101550309472"

3×5×2 Array{Float64,3}:
[:, :, 1] =
  0.360468   0.18333   0.589174   0.0238477   0.176416
  0.681901   0.605349  0.710431   0.475512    0.603602
 -0.258519  -0.485358  0.34821   -0.584916   -0.483823

[:, :, 2] =
 0.285262  0.130112  0.584725  -0.0372785  0.108469
 0.514397  0.377314  0.662549   0.215813   0.384661
 0.395966  0.190946  0.635119   0.0314545  0.186853

"Delta: 17.225495778984293"

3×5×2 Array{Float64,3}:
[:, :, 1] =
 0.567402   1.03415    0.334754  1.41809  0.869492
 0.767443   0.939604   0.620155  1.00953  0.902988
 0.0785073  0.904442  -0.204766  2.19215  0.57233

[:, :, 2] =
 0.496742  1.11788   0.297744  1.68633  0.864478
 0.668394  1.0056    0.480118  1.23548  0.931282
 0.60601   0.963331  0.373467  1.27385  0.830511

"Delta: -18.382762348253127"

3×5×2 Array{Float64,3}:
[:, :, 1] =
  0.316775   0.0465224  0.735181  -0.0965938   0.0962725
  0.659961   0.507054   0.756751   0.343064    0.547053
 -0.312907  -0.577399   0.744663  -0.628724   -0.543068

[:, :, 2] =
 0.238269  -0.0233248  0.748549  -0.17808    0.0211399
 0.478521   0.240962   0.75961    0.0630525  0.303755
 0.353556   0.0566108  0.781142  -0.0927385  0.109091

In [8]:


function generate(model::Main.FHMMs.FHMM, timesteps::Int)
    X = zeros(size(model.W)[1], timesteps)
    # choose initial states based on π
    states = zeros(Int64, model.M, timesteps)
    for m=1:model.M
        idx = sample(weights(model.π[:,m]))
        states[m,1] = Int64(idx)
    end
    for t=1:timesteps    
        mu = zeros(model.D)
        for m=1:model.M
            mu += model.W[:,states[m,t],m]
        end
        n = MvNormal(mu, model.C)
        X[:,t] = rand(n)
        if t < timesteps
            for m=1:model.M
                nsp = model.P[states[m,t],:,m]
                #display(nsp)
                smpl = sample(weights(nsp))
                states[m,t+1] = smpl
            end
        end
    end
    return X,states
end

generate (generic function with 3 methods)

In [ ]:
function sample_states(model::Main.FHMMs.FHMM, timesteps::Int, iter=1000)
    # we need to sample theta, KxMxT
    # θ[1,1,2] represents the probabilities of chain (1) being in state (1) at time 2, 
    # θ[3,1,2] represents the probabilities of chain (1) being in state (3) at time 2, etc
    # this is unconditional wrt the observation
    theta_sampled = zeros(model.K,model.M, timesteps)
    for i=1:iter
        theta_iter = zeros(model.K,model.M,timesteps)
        for m=1:model.M
            s = sample(weights(model.π[:,m]))
            theta_iter[s, m,1] += 1
        end
        for t=2:timesteps
            for m=1:model_actual.M
                lastState = findfirst((t) -> t == 1, theta_iter[:,m,t-1])
                s = sample(weights(model.P[lastState,:,m]))
                theta_iter[s,m,t] += 1
            end
        end
        theta_sampled .+= theta_iter
    end
    theta_sampled ./= iter
    return theta_sampled
end


In [ ]:
function get_state_combinations(M::Int, K::Int)
    state_combinations = []
    m = 1
    stack = []
    while(true)
        if(m == M)
            if(size(stack)[1] != M - 1)
                break
            end
            for s=1:K
                push!(stack, s)
                push!(state_combinations, copy(stack))
                pop!(stack)
            end
            stack[end] +=1
            while(stack[end] > K)
                pop!(stack)
                if(size(stack)[1] == 0)
                    break
                end
                stack[end] +=1
                m -= 1
            end
        else
            push!(stack, 1)
            m += 1
        end
    end
    return state_combinations
end

In [ ]:
# we want to calculate 
# P(Mt=St|Yt)
# = P(Yt|Mt=St).P(Mt=St|Mt-1=St-1) / P(Yt)
# This decoding is specific to the completely factorized VI model which uses a simplified P(Mt|Yt) that assumes 
# no dependence between timesteps. Viterbi decoding is therefore not required, since we don't need to find the max sequence,
# we just take the max P at each timestep.
# even though each chain is independent but the observation is conditional on *all* chains 
# this is because Yt is a sum of M1,M2,M3...
# i.e. P(Yt|M1t=S1t,M2t=.., M3t=..) != P(Yt|M1t=S1t)
# but we have θ, which is an estimation of P(M1t=S1t), P(M1t=S2t)...P(Mmt=Sst)
# so we need to find all configurations of M/S at this t
# cross product of (M1xS)x(M2xS)x..x(MkxS)
# if we are using the variational inference model, θ is KxMxT
# θ[1,1,2] represents p(chain (1) in state (1) at time 2) 
# θ[3,1,2] represents p(chain (1) in state (3) at time 2), etc
# this is unconditional wrt the observation
# i.e. P(Mt=St|M(t-1)=S(t-1))
# therefore P(Mt=St) is the product over dim(1) at T
function decode(model::Main.FHMMs.FHMM, X::Array{Float64,2}, θ::Array{Float64,3})
    
    D=size(X)[1]
    timesteps=size(X)[2]
    
    states = zeros(model.M,timesteps)
    for t=1:timesteps
        pmax = -1
        mmax = []
        stack = []
        m = 1
        M = model.M
        K = model.K
        while(true)
            if(m == M)
                if(size(stack)[1] != M - 1)
                    break
                end
                for s=1:K
                    push!(stack, s)
                    mu = zeros(D)
                    for i=1:M
                        mu += model.W[:, stack[i], i]
                    end
                    
                    p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                    p_mt_in_s = reduce(*, θ[stack,:,t])
                    unnormalized = p_yt_given_mt_in_s * p_mt_in_s
                    if unnormalized > pmax
                        pmax = unnormalized
                        mmax = copy(stack)
                    end
                    pop!(stack)
                end
                stack[end] +=1
                while(stack[end] > K)
                    pop!(stack)
                    if(size(stack)[1] == 0)
                        break
                    end
                    stack[end] +=1
                    m -= 1
                end
            else
                push!(stack, 1)
                m += 1
            end
        end
        states[:,t] = mmax
    end
    return states
end

In [ ]:
function decode_full(model::Main.FHMMs.FHMM, X::Array{Float64,2})
    D=size(X)[1]
    timesteps=size(X)[2]
    state_combinations = get_state_combinations(model.M, model.K)
    state_probabilities = zeros(size(state_combinations)[1],timesteps)
    
    M = model.M
    K = model.K
    logP = log.(model.P)
    for t=1:timesteps
        for s_index=1:size(state_combinations)[1]
            state = state_combinations[s_index]
            mu = zeros(D)
            for i=1:model.M
                mu += model.W[:, state[i], i]
            end
            p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                   
            if(t == 1)
               p_mt_in_s = reduce(*, model.π[[CartesianIndex(state[i], i)  for i=1:model.M]])
            else
                # calculate the probability that chain M is in state K, P(Mt)
                # this is P(Mt|Mt-1, Yt), so P(Mt-1|Yt) * P(Mt|Mt-1)
                # P(Mt-1|Yt) is sum over K
                p_mt_in_s = 0
                for sc_last_index=1:size(state_combinations)[1]
                    sc_last = state_combinations[sc_last_index]
                    for m_last=1:M
                        for m_this=1:M
                            p_mt_in_s += exp(
                                state_probabilities[sc_last_index,t-1] + 
                                logP[sc_last[m_this], state[m_this], m_this])
                        end
                    end
                end
            end
            state_probabilities[s_index,t] = log(p_yt_given_mt_in_s) +  log(p_mt_in_s)
         end
    end
    max_probs = zeros(M, timesteps)
    i = 1
    for p in argmax(state_probabilities, dims=1)
        max_probs[:, p[2]] = state_combinations[p[1]]
        i += 1
    end
    return max_probs
end

#X, actual = generate(model_actual, 2)
#decoded = decode_full(model_actual,X)
#actual .== decoded

In [ ]:
# setup our known model
# 
D=15
K=3
M=2
#T=5
W=cat( [ 1.0 49.0; -1.0 -49.0 ],[ 1.0 49.0; -1.0 -49.0 ], [ 1.0 49.0; -1.0 -49.0 ], dims=3)
π = [ 0.35 0.25 0.95; 0.65 0.75 0.05; ]
P = cat([ 0.3 0.7 ; 0.2 0.8 ; ],
    [ 0.5 0.5 ; 0.4 0.6 ;  ],
    [ 0.75 0.25 ; 0.89 0.11 ; ], dims=3)
W = rand(D,K,M)
π = rand(K,M)
π ./= sum(π,dims=(1))
P = rand(K, K, M)
P ./= sum(P, dims=2)
#C = rand(D,D)
C = Array{Float64}(Diagonal([0.1 for x in 1:D]))
model_actual = Main.FHMMs.FHMM(D,K,M,
    W, π, P, C)

X, actual = generate(model_actual, 4)



In [ ]:
model_learn = Main.FHMMs.FHMM(D,K,M)
result = Main.FHMMs.fit_sv!(model_learn,X,maxiter=5000,verbose=true)    

In [ ]:
result.γ

In [ ]:
actual

In [ ]:
timesteps=10
num_observations = 100

theta_sampled = sample_states(model_actual, timesteps, 1000)
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode(model_actual,X, theta_sampled)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode_full(model_actual,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 50
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100)    
    
    estimated_states = decode_full(model_learn,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
timesteps=10
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 100
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100,fzero=true)    
    for i in 1:D
        model_learn.C[i, i] += 0.0001
    end
    #println(cholesky(Hermitian(Symmetric(model_learn.C))))
    estimated_states = decode(model_learn,X, result.θ)
    j += sum(actual_states .== estimated_states)
    println(j / (i * timesteps * model_actual.M))
end
j / (num_observations * timesteps * model_actual.M)